In [133]:
import itertools
import random
import json
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path 
import sys5573830033355906
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import os
import logging
import threading
import time

In [134]:
YEARS = [str(y) for y in range(1990,2019)]
df_path = 'myDATA/01- france_collaboration_df.csv'
all_jsons = [('Data/COMP/' + 'COMP-'+year+'.json') for year in YEARS] 

In [135]:
# iterate over years
df = pd.DataFrame(columns=np.concatenate((["ID"], YEARS), axis=None))
for i in range(11, len(YEARS)):
    print(YEARS[i])
    year = YEARS[i]
    file_path = all_jsons[i]
    
    f = open(file_path)
    json_file = json.load(f)
    
    seen = {}
    # iterate over publications
    for pub in json_file: 
        for aut in pub['authors']:
            curr_aut = aut['id']
            seen[curr_aut] = set()
            for other_aut in pub['authors']:
                curr_collab = other_aut['id']
                if(curr_aut != curr_collab):
                    seen[curr_aut].add(curr_collab)
    for aut in seen:
        num_collabs = len(seen[aut])
        if((len(df[df['ID']  == aut]) == 0)):
            df = df.append({'ID':aut, year:num_collabs}, ignore_index=True)
        else:
            df.loc[df['ID']  == aut, year] = num_collabs
            


1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001


KeyboardInterrupt: 

In [136]:
df.to_csv(df_path, index=False)